<a href="https://colab.research.google.com/github/korneliuszm/energy_planner/blob/main/Wyznaczanie_godzin_poboru_energii_RDN_OK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [419]:
%reset -f
#Ważne informacje
#Pobór energii przez zmywarkę - 0.47; 0.04; 0.45 kWh
#Pobór energii przez pralkę (40*C, suszenie 1) - 0.5; 0.08; 0.08; 1; 0.92 kWh
washing_machine_energy_consumption = [0.5, 0.08, 0.08, 1, 0.92]
dishwasher_energy_consumption = [0.47, 0.04, 0.45]

day_distribution_price = 0.2817 + 0.00618 + 0.0314 + 0.095 + 0.005
night_distribution_price = 0.0927 + 0.00618 + 0.0314 + 0.095 + 0.005

#Taryfa standardowa Enea
old_day_price = ( (0.5 + 0.3050 + 0.00496 + 0.0095 ) * 1.23 )
old_night_price = ( (0.5 + 0.1004 + 0.00496 + 0.0242 ) * 1.23 )

import requests
import pandas as pd
import json
from datetime import datetime, timedelta
import pytz

# Get the current time in your timezone
your_timezone = pytz.timezone('Europe/Warsaw')  # Replace with your timezone
now = datetime.now(your_timezone)

# Floor the current time to the nearest 0 minutes
floored_time = now.replace(minute=(now.minute // 60), second=0, microsecond=0)

r_today = requests.get('https://energy-instrat-api.azurewebsites.net/api/prices/energy_price_rdn_hourly')

df_today = pd.json_normalize(r_today.json())

In [420]:
combined_df = df_today.copy()

In [421]:
combined_df = df_today.copy()


combined_df['date'] = pd.to_datetime(pd.to_datetime(combined_df['date']))

combined_df = combined_df.drop(['index','fixing_i.volume', 'fixing_ii.price', 'fixing_ii.volume'], axis=1, errors='ignore')
combined_df = combined_df[combined_df['date'].dt.minute == 0].reset_index()

#Przelicz na grosze
combined_df['fixing_i.price'] = combined_df['fixing_i.price']/1000

#Dodanie nowych kolumn
combined_df['dishwasher'] = 0                                                   #0 - nieaktywne, 1 -  najtańszy prąd
combined_df['washing_machine'] = 0                                              #0 - nieaktywne, 1 -  najtańszy prąd
combined_df['water_heating'] = 0                                                #0 - nieaktywne, 1 -  pompa ciepła, 2 - grzałka
combined_df['storage_heater'] = 0                                               #0 - nieaktywne, 1 -  najtańszy prąd
combined_df['car_charging'] = 0                                                 #0 - nieaktywne, 1 -  ładuj
combined_df['current_hour'] = 0
combined_df['old_price'] = 0                                                    #stara cena energii


#Dodanie cen dystrybucji
for index, row in combined_df.iterrows():
  if (6 <= row['date'].hour < 13) or (15 <= row['date'].hour < 22):
    combined_df.loc[index, 'fixing_i.price_with_distribution'] =(row['fixing_i.price'] + day_distribution_price)*1.23
    combined_df.loc[index, 'old_price'] = old_day_price
  else:
    combined_df.loc[index, 'fixing_i.price_with_distribution'] = row['fixing_i.price'] + (night_distribution_price)*1.23
    combined_df.loc[index, 'old_price'] = old_night_price


combined_df['date'] = combined_df['date'].dt.tz_localize(None)
combined_df.reset_index(drop=True, inplace=True)

#kopia do wizualizacji
full_df = combined_df.copy()
full_df.to_csv('full_df.csv', index=False)

#Usunięcie wierszy z przeszłości do przyszłych wyszukiwań
now_localize = pd.to_datetime(floored_time).tz_localize(None)

future_df = combined_df[combined_df['date'] >= now_localize]
future_df.reset_index(drop=True, inplace=True)

if future_df['fixing_i.price'].isnull().any():
  print("Warning: Some cells in the 'cena' column are empty.")

<ipython-input-421-69008958fa13>:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0079357999999998' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined_df.loc[index, 'old_price'] = old_day_price


In [422]:
# Cena dla aktualnej godziny
def actual_fixing_i_price():
  formatted_time = floored_time.strftime('%Y-%m-%d %H:%M')

  try:
    value_rdn_pln_with_distribution = combined_df.loc[combined_df['date'].dt.strftime('%Y-%m-%d %H:%M') == formatted_time, 'fixing_i.price_with_distribution'].iloc[0]
    value_rdn_pln = combined_df.loc[combined_df['date'].dt.strftime('%Y-%m-%d %H:%M') == formatted_time, 'fixing_i.price'].iloc[0]

    return value_rdn_pln, value_rdn_pln_with_distribution
  except IndexError:
    return None

In [423]:
# Znajduje najlepsze godziny dla pralki

def find_best_price_washing_machine():
  best_price_value = 10

  for i in range(len(future_df)-4):
    best_price_value_temp = 0

    for j in range(5):
      best_price_value_temp = best_price_value_temp + future_df.iloc[i+j]['fixing_i.price_with_distribution'] * washing_machine_energy_consumption[j]

    if best_price_value_temp < best_price_value:
      best_price_value = best_price_value_temp
      best_price_index = i

  return best_price_value, best_price_index, future_df.iloc[best_price_index]['date']

In [424]:
# Znajduje najlepsze godziny dla zmywarki

def find_best_price_dishwasher():
  best_price_value = 10

  for i in range(len(future_df)-2):
    best_price_value_temp = 0

    for j in range(3):
      best_price_value_temp = best_price_value_temp + future_df.iloc[i+j]['fixing_i.price_with_distribution'] * dishwasher_energy_consumption[j]

    if best_price_value_temp < best_price_value:
      best_price_value = best_price_value_temp
      best_price_index = i

  return best_price_value, best_price_index, future_df.iloc[best_price_index]['date']

In [425]:
# Dodawanie znaczników w tablicy
def add_markers():
  #Zmywarka
  value, dishwasher_idx, dishwasher_time = find_best_price_dishwasher()

  for i in range(len(combined_df)):
    if combined_df.loc[i, 'date'] == dishwasher_time:
      for j in range(3):
        combined_df.loc[i+j, 'dishwasher'] = 1

  #Pralka
  value, washing_machine_idx, washing_machine_time = find_best_price_washing_machine()

  for i in range(len(combined_df)):
    if combined_df.loc[i, 'date'] == washing_machine_time:
      for j in range(5):
        combined_df.loc[i+j, 'washing_machine'] = 1

  #Ciepła woda
  water_heating_time = future_df.sort_values('fixing_i.price_with_distribution').head(10)

  top_4_prices = water_heating_time['fixing_i.price_with_distribution'].nlargest(4)
  average_top_4 = top_4_prices.mean()

  #Sprawdź czy warto odpalić grzałkę


  for i in range(len(combined_df)):
    if combined_df.loc[i, 'date'] in water_heating_time['date'].values:
      if average_top_4 > 2*water_heating_time.iloc[0]['fixing_i.price_with_distribution']:
        print( "Odpalamy grzałkę na 1 godzinę")
        combined_df.loc[i, 'water_heating'] = 2
      else:
        combined_df.loc[i, 'water_heating'] = 1

  #Grzejniki (jedna najtańsza godzina)
  smallest_prices = future_df.nsmallest(24, 'fixing_i.price_with_distribution')

  # Mark them in the storage_heater column with a number based on the value
  for i, row in smallest_prices.iterrows():
    for j, combined_row in combined_df.iterrows():
      if combined_row['date'] == row['date']:
        combined_df.loc[j, 'storage_heater'] = smallest_prices.index.get_loc(i) + 1  # Assign a number based on the rank

  # Zaznacz aktualną godzinę/datę
  for index, row in combined_df.iterrows():
    if row['date'].hour == now.hour and row['date'].day == now.day:
      combined_df.loc[index, 'current_hour'] = 1

  return dishwasher_idx, washing_machine_idx

In [426]:
# Dodaje znaczniki do tablicy full_df
dishwasher_start, washing_machine_start = add_markers()


In [427]:
#Zapisz cały df do csv
combined_df.to_csv('combined_df.csv', index=False)

#Zapisz czasy do startu urządzeń
data = {
    "dishwasher_start": dishwasher_start,
    "washing_machine_start": washing_machine_start+5
}
start_time_json = json.dumps(data)

#Zapisz json z parametrami aktualnej godziny
current_hour_json = full_df[(full_df['date'].dt.hour == now.hour) & (full_df['date'].dt.day == now.day)].to_json(orient='records')